In [ ]:
import numpy as np # type: ignore
import tensorflow as tf # type: ignore
from PIL import Image # type: ignore

In [47]:
# Muat model TFLite
interpreter = tf.lite.Interpreter(model_path="food_classifier_flex.tflite")
interpreter.allocate_tensors()

In [48]:
# Info input/output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [49]:
# Ukuran input yang diharapkan model
input_shape = input_details[0]['shape']  # contoh: [1, 224, 224, 3]

In [50]:
# Fungsi untuk load dan preprocessing gambar
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((input_shape[1], input_shape[2]))
    input_data = np.expand_dims(image, axis=0).astype(np.float32)
    # Normalisasi (jika model butuh), sesuaikan dengan model kamu!
    input_data = input_data / 255.0
    return input_data

In [51]:
# Fungsi untuk memuat label dari file txt
def load_labels(filename):
    with open(filename, "r") as f:
        return [line.strip() for line in f.readlines()]

In [52]:
# Load label
labels = load_labels("labels.txt")

In [68]:
# Masukkan gambar
input_data = load_image("tonkotsu.jpg")
interpreter.set_tensor(input_details[0]['index'], input_data)

In [69]:
# Jalankan inferensi
interpreter.invoke()

In [70]:
# Ambil hasil output
output_data = interpreter.get_tensor(output_details[0]['index'])[0]
predicted_index = np.argmax(output_data)
predicted_label = labels[predicted_index]
confidence = output_data[predicted_index]

In [71]:
print(f"Prediksi: {predicted_label} (Confidence: {confidence:.2f})")

Prediksi: Ramen (Confidence: 0.89)


In [72]:
# Cetak Top-5
top_k = 5
top_indices = output_data.argsort()[-top_k:][::-1]
for i in top_indices:
    print(f"{labels[i]}: {output_data[i]*100:.2f}%")

Ramen: 89.01%
Bibimbap: 10.56%
Gyoza: 0.05%
Huevos rancheros: 0.04%
Croque madame: 0.03%
